In [2]:
import pandas as pd
import numpy as np

In [3]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")

In [4]:
df_test.head(5)

,free_text,label_id
0,Đừng cố biện minh =)))) choi lon,0
1,Haizz. Nthe này thì dân khổ quá,1
2,the nay ma chi phat gay roi trat tu cong cong ...,0
3,Mua cho em hộp bcs mĩ sài cho oai :)),0
4,coin card :3,1


In [5]:
df_train = df_train.dropna()
df_train.isna().sum()

free_text    0
label_id     0
dtype: int64

In [6]:
df_test = df_test.dropna()
df_test.isna().sum()

free_text    0
label_id     0
dtype: int64

# Hàm làm sạch văn bản

In [7]:
import re
def clean_text(sentence):
    sentence = re.sub(r'[^a-zA-Z0-9À-Ỵà-ỹ\s]', '', sentence)
    sentence = re.sub(r'\s+', ' ', sentence).strip()

    return sentence


df_train["free_text"].apply(clean_text)
df_test["free_text"].apply(clean_text)


0                              Đừng cố biện minh choi lon
1                          Haizz Nthe này thì dân khổ quá
2       the nay ma chi phat gay roi trat tu cong cong ...
3                       Mua cho em hộp bcs mĩ sài cho oai
4                                             coin card 3
                              ...                        
6675                   Nghe cũng được Nhưng vẫn cày views
6676    Pha team up căng cực đến từ vị trí ông chủ đầm...
6677    Thanh Nhường sao m đọc đc cmt của t thế tiên sư m
6678    Đúng rồi nhìn có ria mép với mặt giống đàn ông...
6679    Chưa chắc Mỹ bị nhiễm virus corona Vũ hán của ...
Name: free_text, Length: 6680, dtype: object

# Convert word

## Binary

In [8]:
from sklearn.feature_extraction.text import CountVectorizer


binary_vectorizer =  CountVectorizer(binary=True)
X_train = binary_vectorizer.fit_transform(df_train["free_text"])
y_train = df_train["label_id"]
X_test = binary_vectorizer.transform(df_test["free_text"])
y_test = df_test["label_id"]

# Models

In [ ]:
# Các model
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Naive Bayes (Multinomial)": MultinomialNB(),
    "Naive Bayes (Bernoulli)": BernoulliNB(),
    "Linear SVM": LinearSVC(),
    "Kernel SVM (RBF)": SVC(kernel="rbf"),
    "SGD Classifier": SGDClassifier(max_iter=1000, tol=1e-3),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier()
}

accurr = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = round(accuracy_score(y_test, y_pred),2)
    accurr.append(acc)
    print(f"{name}: {acc:.4f}")

# for name, model in models.items():
#     print("="*60)
#     print(f"Model: {name}")
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)
#     print(classification_report(y_test, y_pred))

Logistic Regression: 0.8657
Naive Bayes (Multinomial): 0.8528
Naive Bayes (Bernoulli): 0.8175
Linear SVM: 0.8579
Kernel SVM (RBF): 0.8552
SGD Classifier: 0.8672
Decision Tree: 0.8281
Random Forest: 0.8560
Gradient Boosting: 0.8548


In [12]:
models_name = [name for name,_ in models.items()]
print(models_name)
print(accurr)

['Logistic Regression', 'Naive Bayes (Multinomial)', 'Naive Bayes (Bernoulli)', 'Linear SVM', 'Kernel SVM (RBF)', 'SGD Classifier', 'Decision Tree', 'Random Forest', 'Gradient Boosting']
[0.8657185628742515, 0.8528443113772455, 0.8175149700598803, 0.857934131736527, 0.8552395209580839, 0.8672155688622755, 0.8281437125748503, 0.8559880239520958, 0.8547904191616766]
